## Combine predictions

In [1]:
# file which llama3 and gemma2 predictions are written to
from pathlib import Path
predictions_root_dir_path = Path("/home/watson_chua/efs/hansard_finetuning/data/predictions")
gpt4_zero_shot_predictions_path = predictions_root_dir_path / Path("gpt4_answers_zero_shot.jsonl")
gpt4_one_shot_predictions_path = predictions_root_dir_path / Path("gpt4_answers_one_shot.jsonl")
llama_predictions_path = predictions_root_dir_path / Path("llama3_8b_lora.jsonl")
gemma_predictions_path = predictions_root_dir_path / Path("gemma2_9b_lora.jsonl")


In [2]:
import json
with gpt4_zero_shot_predictions_path.open('r') as f:
    gpt4_zero_shot_lines = f.readlines()

with gpt4_one_shot_predictions_path.open('r') as f:
    gpt4_one_shot_lines = f.readlines()

with llama_predictions_path.open('r') as f:
    llama_lines = f.readlines()

with gemma_predictions_path.open('r') as f:
    gemma_lines = f.readlines()


gpt4_zero_shot_lines = [json.loads(l) for l in gpt4_zero_shot_lines]
gpt4_one_shot_lines = [json.loads(l) for l in gpt4_one_shot_lines]
llama_lines = [json.loads(l) for l in llama_lines]
gemma_lines = [json.loads(l) for l in gemma_lines]


In [3]:
len(gpt4_zero_shot_lines), len(gpt4_one_shot_lines), len(llama_lines), len(gemma_lines)

(304, 304, 304, 304)

In [7]:
import pandas as pd 
df_combined = pd.DataFrame(gpt4_zero_shot_lines)
df_combined.rename(columns={'gpt4_answer_by_hy_doc': 'gpt4_zero_shot_answer'}, inplace=True)
df_combined['gpt4_one_shot_answer'] = [l['gpt4_answer_by_hy_doc'] for l in gpt4_one_shot_lines]
df_combined['llama3_answer'] = [l['generation'] for l in llama_lines]
df_combined['gemma2_answer'] = [l['generation'] for l in gemma_lines]

In [14]:
# check that they are merged correctly and answer the same question
df_combined[['question', 'gpt4_zero_shot_answer', 'gpt4_one_shot_answer', 'llama3_answer', 'gemma2_answer']].sample(1).iloc[0].tolist()

['Mr Yip Hon Weng asked the Deputy Prime Minister and Minister for Finance whether the Ministry will review the Community Development Council Vouchers scheme to better support larger households by adjusting voucher allocations based on household size. ',
 'The Government acknowledges the unique financial pressures faced by larger households and is committed to ensuring that support measures like the Community Development Council (CDC) Vouchers Scheme are equitable and effective. We are currently conducting a thorough review of the CDC Vouchers Scheme to explore enhancements that could better address the needs of larger households. This review includes considerations for scaled voucher distribution based on household size, supplementary allowances for specific needs, and improvements in accessibility and outreach. Our goal is to refine the scheme to provide more tailored support, ensuring that all households receive adequate assistance. We will continue to assess and adapt our measures 

In [15]:
df_combined.to_csv(predictions_root_dir_path /Path("consolidated_predictions.csv"), index=False)